In [100]:
import json_tricks
import numpy
import random
import os

numpy.random.seed(0)
random.seed(0)


inputs1 = json_tricks.load(open('inputs/inputs1.json'))
inputs2 = json_tricks.load(open('inputs/inputs2.json'))

answer = {}


In [101]:
numpy.random.seed(0)
random.seed(0)

# Implementing Backpropagation package for Numpy

In this project, we will implement the backpropagation algorithm for numpy package.
Your task will be to implement all the methods of the Node class that will be a wrapper for numpy arrays
supporting backpropagation.

Step-by-step, we will implement the following methods:

1. `__init__` (a constructor)
3. `backward` (a recursive mechanism that triggers backpropagation)
4. `__neg__` (negation operator $- x$)
5. `__add__` (addition operator $x + y$)
6. `__mul__` (product operator $x \cdot y$)
7. `__sub__` (substitution operator $x - y$)
8. `__truediv__` (division operator $x / y$)
9. `exp` (exponentiation $\exp(x)$)
10. `sum` (summation $\sum_{k} x_k$)
11. `matmul` (matrix product $XY$)

After that we will use the implemented methods on:
1. Simple graph from the previous task
2. Two-layer neural network

Let's start!

`Node([1]) + Node([2]) -> Node`
`(x + y).backward()`

# Task 1

Implement the `__init__` method for the `Node` class:
- create field `data` and assign it to the `data` argument (we will need it to compute the gradient)
- create field `grad` and assign it to the `None` (it will be used to store the gradient of the node)
- create field `inputs` and assign it to the empty list (it will be used to store the nodes on which the current node depends)
- create field `n_dependents` and assign it to the 0 (it will be used to count the number of nodes that depend on the current node)
- create field `n_grads` and assign it to the 0 (it will be used to count the number of gradients that are coming to the current node)

Implement the method `update_grad` for the `Node` class. The point of this method is to take care of the gradients that are coming from the next nodes and accumulate them:
- firstly, create a deep copy of `grad_output` (just in case): `grad_output = copy.deepcopy(grad_output)`
- increase the `n_grads` counter by 1 as one of the gradients of the dependents is registered
- if `grad_output` is `None` and the size of `data` is 1, assign `grad_output` to the numpy array `[1]` (as we can calculate derivatives only of scalar values without any dependencies)
- if `grad_output` is `None`, raise the `BaseException` with the message "Backpropagation is impossible" (as we can't compute the gradient without the gradient from the next node except for the case of a scalar value)
- if `self.grad` is `None`, assign `grad_output` to the `self.grad` (that means that if there was no gradient before, we will just assign the one that we got from the next node)
- otherwise, add `grad_output` to the `self.grad` (that means that if there was a gradient before, we will add the one that we got from the next node to the accumulated value)

Implement the `backward` method for the `Node` class. This method is the heart of the backpropagation algorithm. And it logic is the following:
- it waits and collects gradients from all the dependents of the current node
- then it calculates the list of gradients for the inputs of the current node using the `backward_step` method (this method we will implement for every operation by hand)
- triggers the `backward` method of the inputs with the calculated gradient in the current node as an argument

As a formula, it looks like this:
$\partial_w {L} = \sum_{k=1}^{n} \partial_w {z_k} \partial_{z_k} {L}$

So, how this works in practice?
- call the `update_grad` method with the `grad_output` argument
- if we have collected all the gradients from the dependents, it is time to calculate the gradients for the inputs of the current node 
- call the `backward_step` method with the `grad_output` argument to calculate the gradients for the inputs
- call the `backward` method of the inputs with the calculated gradient in the current node as an argument
- reset the `n_grads` counter to 0 (actually, this is not necessary as we do backpropagation only once)

Also `backward_step` method should be implemented for basic node too. It should return an empty list as basic node does not have any inputs.

In general, we will sotre inputs to the operation as list of nodes in the `inputs` field. The `backward_step` function should return the corresponding list of gradients for the inputs.

After these steps are done, you can already check that the first test passes successfully

# Task 2

Implement the `__add__` method for the `Node` class and the class `TensorSum` that inherits from `Node`. 
This method enables to use the `+` operator for a pair of objects of `Node` class and means that after we perofrm `z = x + y`, result
of this operation is also a Node that correspond to the sum of two nodes.

The engine for that operation is `TensorSum` class, here you need to:
- initialize the `Node` class with the `None` argument (empty envelope of the node) in the `__init__` method
- in the `__call__` method (this method is used when we perform `z = x + y`):
    - assign the `inputs` fields of the current node to the `[input1, input2]` to store the inputs
    - set `n_dependents` to the sum of `n_dependents` of the inputs (or simply to 2 as we have only 2 inputs)
    - calculate value of `data` field of the current node using the `self.inputs[0].data + self.inputs[1].data` formula
    - set the `grad` field of the current node to `None` as we don't have any gradient yet
    - return the `self` node (so that the result of the operation is the current node)
- in the `backward_step` method:
    - return the list of gradients for the inputs: `[grad_output, grad_output]` as $\partial_x {x + y} = 1$ and $\partial_y {x + y} = 1$


In `Node` class:
- implement the `__add__` method:
    - create a new `TensorSum` object with the sum of the `data` of the current node and the `other` node and return it (`return TensorSum()(self, other)`)

Now your Node class supports the `+` operator. You can check it by runnint test number 2. So you can calculate gradients of expressions like `x + y + x + x + y + y`. That is no much, but that is a good start.

# Task 3 and others

Implement the remaining operations by analogy:
- `__sub__` that will be used when we perform `z = x - y`
- `__mul__` that will be used when we perform `z = x * y`
- `__truediv__` that will be used when we perform `z = x / y`
- `__neg__` that will be used when we perform `z = -x`
- `exp` that will be used when we perform `z = numpy.exp(x)`
- `sum` that will be used when we perform `z = numpy.sum(x)`
- `matmul` that will be used when we perform `z = x @ y`

In all these cases, you should use theoretircal derivatives to implement `backward_step` function for every of these operations.

The hardest one will be `matmul` as it is a matrix multiplication. To implement it, use the following rule of thumb:
- the resulting gradient should have the same shape as data
- the gradients of linear functions are also some linear functions

In [ ]:
import numpy as np
import copy

# Node(np.array([1]))
class Node:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.inputs = []
        self.n_dependents = 0
        self.n_grads = 0

    def update_grad(self, grad_output):
        grad_output = copy.deepcopy(grad_output)
        
        self.n_grads += 1

        if grad_output is None:
            if self.data.size == 1:
                grad_output = np.array([1])
            else:
                raise BaseException("Backpropagation is impossible")

        if self.grad is None:
            self.grad = grad_output
        else:
            self.grad = self.grad + grad_output


    def backward(self, grad_output=None):
        self.update_grad(grad_output)
        
        # if self.n_grads == self.n_dependents:
        input_grads = self.backward_step(self.grad)
            
        for i, input_node in enumerate(self.inputs):
            input_node.backward(input_grads[i])
            
        self.n_grads = 0
            
    def backward_step(self, grad_output=None):
        return []

    # @backprop
    # def backward(self, grad_output=None):


    def __str__(self):
        return f"Node(data={self.data}, grad={self.grad})"

    def __neg__(self):
        return TensorNeg()(self)

    def __add__(self, other):
        return TensorSum()(self, other)

    def __mul__(self, other):
        return TensorMul()(self, other)

    def __sub__(self, other):
        return TensorSub()(self, other)

    def __truediv__(self, other):
        return TensorDiv()(self, other)

    def exp(self):
        return TensorExp()(self)

    def sum(self, axis=None):
        return TensorSumReduce()(self)

    def __matmul__(self, other):
        return TensorMatMul()(self, other)


class TensorSum(Node):
    def __init__(self):
        super().__init__(None)

    def __call__(self, input1, input2):
        self.inputs = [input1, input2]
        self.n_dependenies = 2
        self.data = input1.data + input2.data
        self.grad = None
        return self
    
    def backward_step(self, grad_output=None):
        return [grad_output, grad_output]


class TensorSub(Node):
    def __init__(self):
        super().__init__(None)

    def __call__(self, input1, input2):
        self.inputs = [input1, input2]
        self.n_dependenies = 2
        self.data = input1.data - input2.data
        self.grad = None
        return self

    def backward_step(self, grad_output=None):
        return [grad_output, -grad_output]


class TensorMul(Node):
    def __init__(self):
        super().__init__(None)

    def __call__(self, input1, input2):
        self.inputs = [input1, input2]
        self.n_dependenies = 2
        self.data = input1.data * input2.data
        self.grad = None
        return self

    def backward_step(self, grad_output=None):
        x, y = self.inputs
        return [grad_output * y.data, grad_output * x.data]


class TensorDiv(Node):
    def __init__(self):
        super().__init__(None)

    def __call__(self, input1, input2):
        self.inputs = [input1, input2]
        self.n_dependenies = 2
        self.data = input1.data / input2.data
        self.grad = None
        return self

    def backward_step(self, grad_output=None):
        x, y = self.inputs
        return [grad_output / y.data, -grad_output * x.data / (y.data ** 2)]


class TensorNeg(Node):
    def __init__(self):
        super().__init__(None)

    def __call__(self, input1):
        self.inputs = [input1]
        self.n_dependenies = 1
        self.data = -input1.data
        self.grad = None
        return self

    def backward_step(self, grad_output=None):
        return [-grad_output]


class TensorExp(Node):
    def __init__(self):
        super().__init__(None)

    def __call__(self, input1):
        self.inputs = [input1]
        self.n_dependenies = 1
        self.data = np.exp(input1.data)
        self.grad = None
        return self

    def backward_step(self, grad_output=None):
        return [grad_output * self.data]


class TensorSumReduce(Node):
    def __init__(self):
        super().__init__(None)

    def __call__(self, input1):
        self.inputs = [input1]
        self.n_dependenies = 1
        self.data = np.sum(input1.data)
        self.grad = None
        return self

    def backward_step(self, grad_output=None):
        x = self.inputs[0]
        return [np.ones_like(x.data) * grad_output]


class TensorMatMul(Node):
    def __init__(self):
        super().__init__(None)

    def __call__(self, input1, input2):
        self.inputs = [input1, input2]
        self.n_dependenies = 2
        self.data = input1.data @ input2.data
        self.grad = None
        return self

    def backward_step(self, grad_output=None):
        A, B = self.inputs
        if A.data.ndim > 2 or B.data.ndim > 2:
            grad_A = grad_output @ B.data.swapaxes(-1, -2)
            grad_B = A.data.swapaxes(-1, -2) @ grad_output
        else:
            grad_A = grad_output @ B.data.T
            grad_B = A.data.T @ grad_output
        return [grad_A, grad_B]


In [103]:
# TEST 1

x = Node(numpy.array([1]))
print(x)
x.backward()
print(x)

answer['init'] = []
for inp in inputs1:
    x = Node(inp['x'][0])

    x.backward()

    answer['init'].append(x.grad)

Node(data=[1], grad=None)
Node(data=[1], grad=[1.])


In [104]:
# TEST 2

x = Node(numpy.array([1]))
y = Node(numpy.array([2]))

z = x + y + x + x + x + y

print(z)
z.backward()
print(x, y)

answer['sum'] = []
for inp in inputs1:
    x = Node(inp['x'][0])
    y = Node(inp['x'][1])

    z = x + y + x + x + x + y
    z.backward()

    answer['sum'].append(x.grad)

Node(data=[8], grad=None)
Node(data=[1], grad=[4.]) Node(data=[2], grad=[2.])


In [105]:
# TEST 3

x = Node(numpy.array([1]))
y = Node(numpy.array([2]))

z = x - y + x + x - x - y

print(z)
z.backward()
print(x, y)

answer['diff'] = []
for inp in inputs1:
    x = Node(inp['x'][0])
    y = Node(inp['x'][1])

    z = x - y + x + x - y - y
    z.backward()

    answer['diff'].append(x.grad)

Node(data=[-2], grad=None)
Node(data=[1], grad=[2.]) Node(data=[2], grad=[-2.])


In [106]:
# TEST 4

x = Node(numpy.array([1]))
y = Node(numpy.array([2]))

z = (x + y) * (x - y)

print(z)
z.backward()
print(x, y)

answer['mul'] = []
for inp in inputs1:
    x = Node(inp['x'][0])
    y = Node(inp['x'][1])

    z = (x + y) * (x - y) * (x + x + y)
    z.backward()
    answer['mul'].append(x.grad)

Node(data=[-3], grad=None)
Node(data=[1], grad=[2.]) Node(data=[2], grad=[-4.])


In [107]:
print()
print(answer['mul'])


[array([-6.]), array([-174.]), array([526.]), array([34.]), array([50.]), array([258.]), array([618.]), array([-106.]), array([158.]), array([6.]), array([318.]), array([-58.]), array([72.]), array([-24.]), array([202.]), array([50.]), array([34.]), array([58.]), array([384.]), array([98.]), array([216.]), array([522.]), array([96.]), array([278.]), array([-174.]), array([-184.]), array([54.]), array([162.]), array([18.]), array([158.]), array([54.]), array([54.]), array([24.]), array([78.]), array([58.]), array([408.]), array([54.]), array([526.]), array([414.]), array([34.]), array([-2.]), array([54.]), array([254.]), array([102.]), array([-34.]), array([-78.]), array([202.]), array([442.]), array([-78.]), array([202.]), array([50.]), array([234.]), array([262.]), array([-106.]), array([216.]), array([18.]), array([18.]), array([-58.]), array([-54.]), array([78.]), array([278.]), array([34.]), array([216.]), array([-34.]), array([-72.]), array([-106.]), array([642.]), array([278.]),

In [108]:
# TEST 5

x = Node(numpy.array([1]))
y = Node(numpy.array([2]))

z = x / y

print(z)
z.backward()
print(x, y)

answer['div'] = []
for inp in inputs1:
    x = Node(inp['x'][0])
    y = Node(inp['x'][1])

    z = x / (Node(0.5) + y)
    z.backward()

    answer['div'].append(x.grad)

Node(data=[0.5], grad=None)
Node(data=[1], grad=[0.5]) Node(data=[2], grad=[-0.25])


In [109]:
# TEST 6

x = Node(numpy.array([1]))
y = Node(numpy.array([2]))

z = -x

print(z)
z.backward()
print(x)

answer['neg'] = []
for inp in inputs1:
    x = Node(inp['x'][0])

    z = -x
    z.backward()

    answer['neg'].append(x.grad)

Node(data=[-1], grad=None)
Node(data=[1], grad=[-1.])


In [110]:
# TEST 7

x = Node(numpy.array([1]))
y = Node(numpy.array([2]))

z = (x + y).exp()

print(z)
print(x.grad, y.grad)

z.backward()

print(x.grad, y.grad)

answer['exp'] = []
for inp in inputs1:
    x = Node(inp['x'][0])

    z = x.exp()
    z.backward()

    answer['exp'].append(x.grad)

Node(data=[20.08553692], grad=None)
None None
[20.08553692] [20.08553692]


# Task

Implement Graph function from the previous task. For the constants please use `Node(1)` whenever needed (otherwise you will get an error)

In [ ]:
# TEST 8 (Graph)

def sigmoid(z):
    return Node(1.0) / (Node(1.0) + (-z).exp())

def tanh(z):
    return (z.exp() - (-z).exp()) / (z.exp() + (-z).exp())

def graph_value(x, w):
    x1, x2 = x[0], x[1]
    b1, b2 = w[0], w[1]
    c1, c2 = w[2], w[3]

    z1 = x1 + b1
    z2 = x2 + b2
    z3 = sigmoid(z1)
    z4 = sigmoid(z2)
    z5 = tanh(z2)
    z6 = z5 * c2
    z7 = z1 * z4
    z8 = z7 * c1
    z9 = z6 * z3
    y = z9 + z8

    return y

answer['graph'] = []
for inp in inputs1:

    x = inp['x']
    w = inp['w']

    x = [Node(float(val)) for val in x]
    w = [Node(float(val)) for val in w]

    y = graph_value(x, w)
    y.backward()

    answer['graph'].append([x[0].grad, x[1].grad, w[0].grad, w[1].grad, w[2].grad, w[3].grad])

print("OUR RESULTS:")
print(x[0].grad, x[1].grad, w[0].grad, w[1].grad, w[2].grad, w[3].grad)

# def torch_graph_value(x, w):
    # return torch.exp(x + w)

# import torch
# x = torch.tensor(inp['x'], requires_grad=True, dtype=float)
# w = torch.tensor(inp['w'], requires_grad=True, dtype=float)

# y = torch_graph_value(x, w)
# y.backward()

# print("TORCH RESULTS:")
# print(x.grad, w.grad)
    

OUR RESULTS:
[2.99906768] [-0.01199483] [-0.01199483] [2.99906768] [-11.9959758] [6.14417322e-06]


# 2-layer NN

Implement 2 layer Neural Network and compute its gradient using `Node` class:

$\mathbf y = \sigma( W_2 \sigma(W_1 \mathbf x + \mathbf b_1) + \mathbf b_2)$

Return sum of all values in $y * y$ as loss function

In [ ]:
# TEST 9 (Two-layer net)

def two_layer_net(x, W1, W2, b1, b2):
    inner = sigmoid(W1 @ x + b1)
    y = sigmoid(W2 @ inner + b2)
    return (y * y).sum()


answer['two_layer_net'] = []
for inp in inputs2:
    x = Node(inp['x'])
    W1 = Node(inp['W1'])
    W2 = Node(inp['W2'])
    b1 = Node(inp['b1'])
    b2 = Node(inp['b2'])

    h_hat = two_layer_net(x, W1, W2, b1, b2)
    h_hat.backward()

    answer['two_layer_net'].append([x.grad, W1.grad, W2.grad, b1.grad, b2.grad])


# Conclusion

You have implemented a backpropagation algorithm. This algorithm is similar to one that is used in Torch. Note that you have implemented all the mechanics of it. Thus it should be now not a magical box: you know exactly how it works.

In [ ]:
json_tricks.dump(answer, '.answer.json')

'{"init": [{"__ndarray__": [1.0], "dtype": "float64", "shape": [1]}, {"__ndarray__": [1.0], "dtype": "float64", "shape": [1]}, {"__ndarray__": [1.0], "dtype": "float64", "shape": [1]}, {"__ndarray__": [1.0], "dtype": "float64", "shape": [1]}, {"__ndarray__": [1.0], "dtype": "float64", "shape": [1]}, {"__ndarray__": [1.0], "dtype": "float64", "shape": [1]}, {"__ndarray__": [1.0], "dtype": "float64", "shape": [1]}, {"__ndarray__": [1.0], "dtype": "float64", "shape": [1]}, {"__ndarray__": [1.0], "dtype": "float64", "shape": [1]}, {"__ndarray__": [1.0], "dtype": "float64", "shape": [1]}, {"__ndarray__": [1.0], "dtype": "float64", "shape": [1]}, {"__ndarray__": [1.0], "dtype": "float64", "shape": [1]}, {"__ndarray__": [1.0], "dtype": "float64", "shape": [1]}, {"__ndarray__": [1.0], "dtype": "float64", "shape": [1]}, {"__ndarray__": [1.0], "dtype": "float64", "shape": [1]}, {"__ndarray__": [1.0], "dtype": "float64", "shape": [1]}, {"__ndarray__": [1.0], "dtype": "float64", "shape": [1]}, {"_